# Arknights CV Web Scraping

<b>Author:<b/> <a href="https://github.com/sdkvictor">sdkvictor<a/>

<b>Last update:<b/> 6/26/2022

In [1]:
!pip install -U selenium

In [2]:
!pip install webdriver-manager

In [3]:
!pip install pandas

## Setup for Selenium

In [4]:
import selenium
selenium.__version__

'4.7.2'

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "C:/Program Files/Google/Chrome Beta/Application/chrome.exe" #CHANGE THIS TO YOUR CHROME PATH

In [7]:
service = Service(executable_path="C:/chromedriver.exe") #CHANGE THIS TO YOUR CHROMEDRIVER EXE PATH
driver = webdriver.Chrome(options=chrome_options, service=service)

## Initializing the Dataframe

In [8]:
import pandas as pd
from selenium.webdriver.common.by import By

In [9]:
#Create the dataframe
df = pd.DataFrame(columns = ['CV', 'Roles', 'Other Roles'])

In [10]:
page_urls = ['https://arknights.fandom.com/wiki/Character_voices/Japanese/A-I', 'https://arknights.fandom.com/wiki/Character_voices/Japanese/J-Q', 'https://arknights.fandom.com/wiki/Character_voices/Japanese/R-Z']

## Getting data from page table

In [11]:
# Access each page 
for url in page_urls:    
    driver.get(url)
    #Get the list of cvs
    cv_list = driver.find_element(By.CLASS_NAME, "mrfz-wtable")
    #Extract the values of the cvs list
    cvs = cv_list.find_elements(By.TAG_NAME, "tr")
    for cv in cvs[1:]:
        columns = cv.find_elements(By.TAG_NAME, "td")
        cv_name = columns[0]
        cv_roles_cell = columns[1]
        cv_roles = cv_roles_cell.find_elements(By.TAG_NAME,"span")
        show_btn = cv_roles[-1]
        if len(cv_roles) > 1:
            cv_roles = cv_roles[:-1]
        cv_name_en = cv_name.find_element(By.TAG_NAME, 'rt').text
        cv_roles_names = []
        for role in cv_roles:
            cv_roles_names.append(role.get_attribute('data-name'))
        cv_roles_list = ', '.join(cv_roles_names)
        try:
            show_btn.send_keys(selenium.webdriver.common.keys.Keys.SPACE)
            cv_other_roles = cv_roles_cell.find_elements(By.TAG_NAME, 'li')
            cv_other_roles_names = []
            for role in cv_other_roles:
                cv_other_roles_names.append(role.text)  
            cv_other_roles_list = ', '.join(cv_other_roles_names)
        except:
            cv_other_roles_list = ""
        df_temp = pd.DataFrame({'CV': cv_name_en, 'Roles': [cv_roles_list], 'Other Roles': [cv_other_roles_list]})
        df = pd.concat([df, df_temp], ignore_index = True)

In [12]:
df

,CV,Roles,Other Roles
0,Abe Atsushi,"Castle-3, Spot","Inojin Yamanaka (Boruto), Moritaka Mashiro (Ba..."
1,Aida Rikako,Quartz,"Riko Sakurauchi (Love Live! Sunshine!!), Yuna ..."
2,Aihara Kotomi,Erato,Shiki Ichinose (The iDOLM@STER Cinderella Girl...
3,Akao Hikaru,Bubble,"Kaoruko Moeta (Comic Girls), Koharu Shimoe (Bl..."
4,Akesaka Satomi,Aurora,"Esdeath (Akame ga Kill!), Chrome Dokuro (Katek..."
...,...,...,...
205,Yukana,Dusk,"C.C. (Code Geass), Tear Grants (Tales of the A..."
206,Yūki Aoi,"Earthspirit, Istina, Nian","Madoka Kaname (Puella Magi Madoka Magica), Tan..."
207,Yūki Kana,"Gitano, Myrrh","Noelle Silva, Acier Silva (Black Clover), Supe..."
208,Yukino Satsuki,Justice Knight,"Kagome Higurashi (Inuyasha), Nenene Sumiregawa..."


In [13]:
# backup
initial_df = df.copy()

In [14]:
len(df)

210

In [15]:
#Save csv
df.to_csv('arknights-japanese-cvs.csv')